In [ ]:
import os, sys
import networkx as nx
import network_analysis

os.chdir(os.path.expanduser('~/capstone-project'))
sys.path.append('src')

In [ ]:
amoxicillin_G = nx.read_graphml('results/Escherichia_coli_amoxicillin/Escherichia_coli_amoxicillin_LOR_annotated.graphml')
ampicillin_G = nx.read_graphml('results/Escherichia_coli_ampicillin/Escherichia_coli_ampicillin_LOR_annotated.graphml')
cefalothin_G = nx.read_graphml('results/Escherichia_coli_cefalothin/Escherichia_coli_cefalothin_LOR_annotated.graphml')
ciprofloxacin_G = nx.read_graphml('results/Escherichia_coli_ciprofloxacin/Escherichia_coli_ciprofloxacin_LOR_annotated.graphml')
levofloxacin_G = nx.read_graphml('results/Escherichia_coli_levofloxacin/Escherichia_coli_levofloxacin_LOR_annotated.graphml')
streptomycin_G = nx.read_graphml('results/Escherichia_coli_streptomycin/Escherichia_coli_streptomycin_LOR_annotated.graphml')
sulfamethoxazole_G = nx.read_graphml('results/Escherichia_coli_sulfamethoxazole/Escherichia_coli_sulfamethoxazole_LOR_annotated.graphml')
tetracycline_G = nx.read_graphml('results/Escherichia_coli_tetracycline/Escherichia_coli_tetracycline_LOR_annotated.graphml')
trimethoprim_G = nx.read_graphml('results/Escherichia_coli_trimethoprim/Escherichia_coli_trimethoprim_LOR_annotated.graphml')

We will cluster the network into communities, ine ach community we will check for the LOR of resistance distribution. We will thus calculate the standard deviation of it in the community and compare it with the standard deviation of the whole network. 

In [ ]:
G = nx.read_graphml('results/merged_network.graphml')

In [ ]:
network_analysis.plot_degree_distribution(G,'pan network')
network_analysis.plot_log_log(G,'pan network')

In [ ]:
# check if G is irected
print(nx.is_directed(G))
# make it undirected
G = nx.Graph(G)
print(nx.is_directed(G))

In [ ]:
# get the max and min edge weights and their respective edge tuple
max_edge = max(G.edges(data=True), key=lambda x: x[2]['weight'])
min_edge = min(G.edges(data=True), key=lambda x: x[2]['weight'])
print(max_edge)
print(min_edge)

In [ ]:
network_analysis.compute_network_stats(G,'pan network')

### amoxicillin

In [ ]:
# --louvain
drug='tetracycline'
G= nx.read_graphml(f'results/Escherichia_coli_{drug}/Escherichia_coli_{drug}_LOR_annotated.graphml')

mean_LOR=sum([G.nodes[node][f'log_odds_{drug}_resistance'] for node in G.nodes])/len(G.nodes)
std_dev_LOR=sum([(G.nodes[node][f'log_odds_{drug}_resistance']-mean_LOR)**2 for node in G.nodes])**0.5

communities=nx.algorithms.community.louvain_communities(G, seed=42)

node_count=[]
new_communities=[]
for i in range(len(communities)):
    node_count.append(len(communities[i]))
    if len(communities[i])>50:
        new_communities.append(communities[i])

import matplotlib.pyplot as plt
plt.hist(node_count, bins=50)
plt.xlabel('Number of Nodes in Community')
plt.ylabel('Number of Communities')
plt.title(f'Number of nodes in {len(communities)} communities for {drug}')

distributions={}; i=0
for c in new_communities:
    community_LOR=[]
    for node in c:
        community_LOR.append(G.nodes[node][f'log_odds_{drug}_resistance'])
    # print(community_LOR)
    mean=sum(community_LOR)/len(community_LOR)
    std_dev=sum([(x-mean)**2 for x in community_LOR])**0.5
    distributions[i]=(mean,std_dev)
    i+=1

In [ ]:
print(f'there are {len(distributions)} communities with more than 50 nodes')
print(distributions)

In [ ]:
# performing  Kolmogorov-Smirnov (KS) test to compare the distributions of LOR values for the communities
from scipy.stats import kstat

for i in range(len(distributions)):
    

In [ ]:
drug='amoxicillin'
G= nx.read_graphml(f'results/Escherichia_coli_{drug}/Escherichia_coli_{drug}_LOR_annotated.graphml')

mean_LOR=sum([G.nodes[node][f'log_odds_{drug}_resistance'] for node in G.nodes])/len(G.nodes)
std_dev_LOR=sum([(G.nodes[node][f'log_odds_{drug}_resistance']-mean_LOR)**2 for node in G.nodes])**0.5

communities=nx.algorithms.community.girvan_newman(G)
communities = list(communities)

distributions={}; i=1
for c in communities:
    community_LOR=[]
    for node in c:
        community_LOR.append(G.nodes[node][f'log_odds_{drug}_resistance'])
    mean=sum(community_LOR)/len(community_LOR)
    std_dev=sum([(x-mean)**2 for x in community_LOR])**0.5
    distributions[i]=(mean,std_dev)
    i+=1

node_count=[]
for i in range(len(communities)):
    node_count.append(len(communities[i]))

import matplotlib.pyplot as plt
plt.hist(node_count, bins=50)
plt.xlabel('Number of Nodes in Community')
plt.ylabel('Number of Communities')
plt.title(f'Number of nodes in {len(communities)} communities for {drug}')